# PTT 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 印出最新文章的「作者」「標題」「時間」
* ② 印出第一頁所有文章的「作者」「標題」「時間」


In [2]:
import requests
import os
import re
from bs4 import BeautifulSoup
import pandas as pd
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

### ① 印出最新文章的「作者」「標題」「時間」

In [4]:
for a in range(39086, 39085, -1):
    url = 'https://www.ptt.cc/bbs/Gossiping/index{page}.html'.format(page = a)
    resp = requests.get(url, cookies={'over18': '1'}, headers = headers)
    soup = BeautifulSoup(resp.text)

    for i in range(len(soup.find_all("div", class_="r-ent"))-1, -1, -1):
        if soup.find_all("div", class_="r-ent")[i].a == None:
            title = soup.find_all("div", class_="r-ent")[i].find("div", class_ = "title").string.strip()
        else:
            title = soup.find_all("div", class_="r-ent")[i].a.string

        print(title)
        push = soup.find_all("div", class_="r-ent")[i].span
        push = '0' if push == None else str(push.string)

        
        date = soup.find_all("div", class_="date")[i].string
        print(date)
        author = soup.find_all("div", class_ = "author")[i].string
        print(author)
        if soup.find_all("div", class_ = 'title')[i].a != None:
            href = "https://www.ptt.cc" + soup.find_all("div", class_ = 'title')[i].a.get('href')
        else:
            href = "-"
        temp = (pd.DataFrame({'push': push, 'title': title, 'date': date, 'author': author, 'href': href}, index = [0]))
        # ptt_df = ptt_df.append(temp, ignore_index=True)


[新聞] 「我殺了你媽媽」賣場老闆疑殺妻　畏罪
12/12
osuki
[問卦] 請問廣播的功能鍵是哪個
12/12
m16855
Re: [問卦] 調整後的核心仍有問題導致SSH會非常卡??
12/12
emperor
Re: [新聞] 快訊/遠航張綱維傳遺書 只求以死謝罪
12/12
cka
[問卦] 韓國節目配樂是中華民國國歌？
12/12
pecko
[問卦] 調整後的核心仍有問題導致SSH會非常卡??
12/12
kenbo
[問卦] 最值錢的廣播什麼顏色的
12/12
choosing
[新聞] 遠航停飛》自稱遭4人詐騙虧損9億 張綱維
12/12
joe19990815
[新聞] 柯文哲指2024成對手 侯友宜：好朋友麥相害
12/12
saisai9230
[問卦] 遠航薪水那麼低也停業了，有卦嗎？
12/12
michael14
[問卦] 買極新的二手車484還不錯？
12/12
flower42
[問卦] 批踢踢終於修好了
12/12
ququ88005
(本文已被刪除) [n1988771126]
12/12
-
[新聞] 男疑遭神明附身講天語下指示國際化　大鬧
12/12
vdml
Re: [新聞] 被指控用網軍 蔡英文：年輕人不是可以被
12/12
david190
[問卦] 投票會遇到國小同學嗎
12/12
ss2121647
Re: [新聞] 尹衍樑36歲兒代理南山董事長 金管會要面
12/12
jeffwang19
Re: [新聞] 快訊/遠航張綱維傳遺書 只求以死謝罪
12/12
twin1949tw
Re: [爆卦] 遠東航空確定倒閉
12/12
soulboy330
[新聞] 快訊／遠航張綱維傳遺書「只求以死謝罪」
12/12
ken0619


### ② 印出第一頁所有文章的「作者」「標題」「時間」

In [5]:
def get_last_pg(board):
    regex = '\d+'
    pattern = re.compile(regex)
    url = 'https://www.ptt.cc/bbs/{board}/index.html'.format(board = board)
    resp = requests.get(url, cookies = {'over18': '1'}, headers = headers)
    soup = BeautifulSoup(resp.text)
    lastpg = soup.find(class_ = 'btn-group btn-group-paging').find_all("a")[1]['href']
    return int(re.search(pattern, lastpg).group(0)) + 1

In [6]:
get_last_pg('gossiping')

39114

In [47]:
def PTTcrawl(board, back, pushlevel = None):
    ptt_df = pd.DataFrame()
    latest = get_last_pg(board)
    while back > 0:
        url = 'https://www.ptt.cc/bbs/{board}/index{page}.html'.format(board = board, page = latest)
        resp = requests.get(url, cookies={'over18': '1'}, headers = headers)
        soup = BeautifulSoup(resp.text)

        for i in range(len(soup.find_all("div", class_="r-ent"))-1, -1, -1):
            if soup.find_all("div", class_="r-ent")[i].a == None:
                title = soup.find_all("div", class_="r-ent")[i].find("div", class_ = "title").get_text().strip()
            else:
                title = soup.find_all("div", class_="r-ent")[i].a.get_text()

            push = soup.find_all("div", class_="r-ent")[i].span
            push = '0' if push == None else str(push.string)
            if 'X' in push or 'e' in push and pushlevel != None:
                continue
            if push != '爆' and pushlevel != None:
                if int(push) < pushlevel:
                    continue

            date = soup.find_all("div", class_="date")[i].string
            author = soup.find_all("div", class_ = "author")[i].string
            if soup.find_all("div", class_ = 'title')[i].a != None:
                href = "https://www.ptt.cc" + soup.find_all("div", class_ = 'title')[i].a.get('href')
            else:
                href = "-"
            temp = (pd.DataFrame({'push': push, 'title': title, 'date': date, 'author': author, 'href': href}, index = [0]))
            ptt_df = ptt_df.append(temp, ignore_index=True)
        back -= 1
        latest -= 1
    return ptt_df

In [48]:
PTTcrawl("gossiping", 6)

,push,title,date,author,href
0,1,協詢12/12 新北三重大智路與忠孝路口行車記錄器,12/12,ianian23,https://www.ptt.cc/bbs/Gossiping/M.1576133906.A.F04.html
1,33,[公告] 一劍無悔，十二月置底閒聊文,12/02,Bignana,https://www.ptt.cc/bbs/Gossiping/M.1575299597.A.058.html
2,10,協尋12/6台中市向上路往精科五路行車記錄器,12/08,Amybook,https://www.ptt.cc/bbs/Gossiping/M.1575739795.A.1DB.html
3,8,[協尋] 11/20高市大順一路548號順明街行車記錄,11/24,witty,https://www.ptt.cc/bbs/Gossiping/M.1574602085.A.66D.html
4,0,[公告] 八卦板板規(2019.08.21),8/21,arsonlolita,https://www.ptt.cc/bbs/Gossiping/M.1566347622.A.9C7.html
5,0,Re: [新聞] 「民進黨媒體壟斷太嚴重」 于美人：幾乎,12/13,TouchAgain,https://www.ptt.cc/bbs/Gossiping/M.1576172583.A.B6A.html
6,2,Re: [問卦] 2019最經典暴雷的八卦?,12/13,sean12345678,https://www.ptt.cc/bbs/Gossiping/M.1576172383.A.F97.html
7,0,Re: [問卦] 金門是不是真的蠻可憐的,12/13,iammatrix,https://www.ptt.cc/bbs/Gossiping/M.1576172316.A.98B.html
8,0,Re: [問卦] 鳥會飛一飛摔死嗎？,12/13,tsisni,https://www.ptt.cc/bbs/Gossiping/M.1576172310.A.E00.html
9,1,Re: [爆卦] 謝龍介不請雞排了....,12/13,cruelgentle,https://www.ptt.cc/bbs/Gossiping/M.1576172033.A.C35.html


### ③ 試著爬爬看其他版的文章

In [9]:
PTTcrawl("beauty", 20, 70)

,push,title,date,author,href
0,73,[公告] 不願上表特 ＆ 優文推薦 ＆ 檢舉建議專區,5/21,hateOnas,https://www.ptt.cc/bbs/Beauty/M.1558415952.A.8D7.html
1,99,(本文已被刪除) [teramars],12/11,-,-
2,爆,[正妹] 乾 真大,12/10,AmedRosario,https://www.ptt.cc/bbs/Beauty/M.1575954797.A.9E3.html
3,爆,[正妹] 我學妹,12/09,hk5566,https://www.ptt.cc/bbs/Beauty/M.1575900447.A.078.html
4,93,[正妹] NTU大學T,12/09,G2S,https://www.ptt.cc/bbs/Beauty/M.1575885757.A.371.html
5,82,[正妹] 18歲 32JJ cup,12/09,AmedRosario,https://www.ptt.cc/bbs/Beauty/M.1575866178.A.247.html
6,71,[正妹] 在俄羅斯待了三年....Part2,12/07,toykoya,https://www.ptt.cc/bbs/Beauty/M.1575724110.A.4C9.html
7,爆,[正妹] 齋藤飛鳥,12/06,A816,https://www.ptt.cc/bbs/Beauty/M.1575610934.A.CF3.html
8,92,[正妹] 她只是個孩子阿,12/05,AmedRosario,https://www.ptt.cc/bbs/Beauty/M.1575513489.A.702.html
9,78,[正妹] 主播下班後,12/04,airgp32002,https://www.ptt.cc/bbs/Beauty/M.1575467084.A.EF9.html


In [60]:
url = 'https://www.ptt.cc/bbs/Gossiping/search?q=%40'
resp = requests.get(url, cookies = {'over18': '1'})
soup = BeautifulSoup(resp.text)

In [61]:
title = soup.find_all(class_="r-ent")[0].find(class_ = "title").a.get_text()

title

'Re: [問卦] ~(@[email\xa0protected])~ 擁有3000個PTT帳號能幹嘛？'

In [62]:
push = soup.find_all("div", class_="r-ent")[i].span.string
str(push)

'5'